In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './output/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('./input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))

In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('FM')]
len(files)

17

In [6]:
files

['FM_Control_BMnoisyp4.rds',
 'FM_BROCKMAN_BMnoisyp4.rds',
 'FM_Cusanovich2018_BMnoisyp4.rds',
 'FM_cisTopic_BMnoisyp4.rds',
 'FM_chromVAR_BMnoisyp4_kmers.rds',
 'FM_chromVAR_BMnoisyp4_motifs.rds',
 'FM_chromVAR_BMnoisyp4_kmers_pca.rds',
 'FM_chromVAR_BMnoisyp4_motifs_pca.rds',
 'FM_GeneScoring_BMnoisyp4.rds',
 'FM_GeneScoring_BMnoisyp4_pca.rds',
 'FM_Cicero_BMnoisyp4.rds',
 'FM_Cicero_BMnoisyp4_pca.rds',
 'FM_SnapATAC_BMnoisyp4.rds',
 'FM_Scasat_BMnoisyp4.rds',
 'FM_scABC_BMnoisyp4.rds',
 'FM_SCRAT_BMnoisyp4.rds',
 'FM_SCRAT_BMnoisyp4_pca.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file.split('_')
    method = file_split[1]
    dataset = file_split[2].split('.')[0]
    if(len(file_split)>3):
        method = method + '_' + '_'.join(file_split[3:]).split('.')[0]
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

Control


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 8 at resolution 1.5
step 1
got 5 at resolution 0.75
step 2
got 6 at resolution 1.125
BROCKMAN


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 11 at resolution 1.5
step 1
got 5 at resolution 0.75
step 2
got 8 at resolution 1.125
step 3
got 7 at resolution 0.9375
step 4
got 6 at resolution 0.84375
Cusanovich2018


...

step 0
got 21 at resolution 1.5
step 1
got 3 at resolution 0.75
step 2
got 8 at resolution 1.125
step 3
got 4 at resolution 0.9375
step 4
got 4 at resolution 1.03125
step 5
got 6 at resolution 1.078125
SCRAT


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 8 at resolution 1.5
step 1
got 5 at resolution 0.75
step 2
got 6 at resolution 1.125
SCRAT_pca


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 10 at resolution 1.5
step 1
got 5 at resolution 0.75
step 2
got 8 at resolution 1.125
step 3
got 6 at resolution 0.9375


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
Control,0.779759,0.780828,0.790929,0.848678,0.845885,0.846218,0.84847,0.846676,0.846452
BROCKMAN,0.561328,0.555279,0.602035,0.685324,0.680493,0.717159,0.682041,0.681051,0.709716
Cusanovich2018,0.992017,0.768445,0.978179,0.988867,0.891575,0.971112,0.98893,0.863883,0.971277
cisTopic,0.953195,0.947545,0.917894,0.948382,0.94389,0.918501,0.94868,0.944194,0.918896
chromVAR_kmers,0.495589,0.497999,0.531637,0.604726,0.636153,0.651479,0.587439,0.626889,0.641111
chromVAR_motifs,0.398901,0.410407,0.316593,0.553781,0.567098,0.513624,0.555722,0.569391,0.503448
chromVAR_kmers_pca,0.601049,0.596801,0.558717,0.720589,0.700458,0.668644,0.712038,0.701502,0.666186
chromVAR_motifs_pca,0.400536,0.345739,0.393151,0.553189,0.574613,0.532386,0.555077,0.561501,0.532387
GeneScoring,0.00806609,0.35607,0.246211,0.00920484,0.436762,0.324887,0.0148086,0.409791,0.291086
GeneScoring_pca,0.323092,0.357345,0.362712,0.417752,0.438917,0.439756,0.40738,0.428261,0.427891
